In [1]:
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local[*]', 'pyspark')
import numpy

In [2]:
test_rdd = sc.textFile("test_no_header.csv")
test_users=test_rdd.collect()



In [3]:
from Similarities import sim_pearson

rdd = sc.textFile("train_no_header.csv")
rdd = rdd.map( lambda x: x.split(',') )
rdd = rdd.map(lambda x:(x[0],[x[1],int(x[2])]))

rdd = rdd.groupByKey()
rdd = rdd.map(lambda x:(x[0],list(x[1])))

#creating an rdd of test_users to select similarities only among them
test_rdd=rdd.filter(lambda x: (x[0] in test_users))
#computing similarities
sim_rdd=rdd.cartesian(test_rdd)
sim_rdd=sim_rdd.map(lambda x: (x[0][0],[x[1][0],sim_pearson(x[1][1],x[0][1])])).filter(lambda x:x[1][1]>0 and x[1][0]!=x[0])
sim_rdd.takeOrdered(10)

[(u'1', [u'1067', 1.0]),
 (u'1', [u'11867', 0.970725343394151]),
 (u'1', [u'13338', 0.9983374884595829]),
 (u'1', [u'14208', 1.0]),
 (u'1', [u'14794', 1.0]),
 (u'1', [u'3020', 1.0]),
 (u'1', [u'365', 0.1414213562373095]),
 (u'1', [u'4386', 0.7559289460184544]),
 (u'1', [u'5610', 1.0]),
 (u'1', [u'5736', 1.0])]

In [4]:
urm_rdd=(sc
         .textFile("train_no_header.csv")
         .map(lambda x: x.split(','))
         .map(lambda x:(x[0],[x[1],int(x[2])])))


urm_rdd.takeOrdered(10)


[(u'1', [u'11618', 1]),
 (u'1', [u'14227', 5]),
 (u'1', [u'16571', 10]),
 (u'1', [u'19022', 9]),
 (u'1', [u'19478', 4]),
 (u'1', [u'19629', 8]),
 (u'1', [u'20484', 9]),
 (u'1', [u'21101', 10]),
 (u'1', [u'21195', 2]),
 (u'1', [u'22932', 9])]

In [5]:
join_rdd=sim_rdd.join(urm_rdd)
join_rdd.takeOrdered(10)

[(u'1', ([u'1067', 1.0], [u'11618', 1])),
 (u'1', ([u'1067', 1.0], [u'14227', 5])),
 (u'1', ([u'1067', 1.0], [u'16571', 10])),
 (u'1', ([u'1067', 1.0], [u'19022', 9])),
 (u'1', ([u'1067', 1.0], [u'19478', 4])),
 (u'1', ([u'1067', 1.0], [u'19629', 8])),
 (u'1', ([u'1067', 1.0], [u'20484', 9])),
 (u'1', ([u'1067', 1.0], [u'21101', 10])),
 (u'1', ([u'1067', 1.0], [u'21195', 2])),
 (u'1', ([u'1067', 1.0], [u'22932', 9]))]

In [6]:
adjusted_join_rdd=join_rdd.map(lambda x: (x[1][0][0],([x[0],x[1][0][1]],[x[1][1][0],x[1][1][1]])))

In [7]:
adjusted_join_rdd.takeOrdered(10)

[(u'100', ([u'419', 1.0], [u'10542', 5])),
 (u'100', ([u'419', 1.0], [u'10825', 5])),
 (u'100', ([u'419', 1.0], [u'11417', 1])),
 (u'100', ([u'419', 1.0], [u'11940', 5])),
 (u'100', ([u'419', 1.0], [u'12048', 5])),
 (u'100', ([u'419', 1.0], [u'12255', 7])),
 (u'100', ([u'419', 1.0], [u'13049', 4])),
 (u'100', ([u'419', 1.0], [u'13190', 4])),
 (u'100', ([u'419', 1.0], [u'14328', 9])),
 (u'100', ([u'419', 1.0], [u'14381', 8]))]

In [9]:
rec_rdd=(adjusted_join_rdd
         .map(lambda x: ((x[0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
         .groupByKey()
         .map(lambda x: (x[0],list(x[1]))))



In [10]:
rec_rdd.takeOrdered(10)

[((u'100', u'10542'), [(1.0, 5)]),
 ((u'100', u'10825'), [(1.0, 5)]),
 ((u'100', u'11417'), [(1.0, 1)]),
 ((u'100', u'11940'), [(1.0, 5)]),
 ((u'100', u'12048'), [(1.0, 5)]),
 ((u'100', u'12255'), [(1.0, 7)]),
 ((u'100', u'13049'), [(1.0, 4)]),
 ((u'100', u'13190'), [(1.0, 4)]),
 ((u'100', u'14328'), [(1.0, 9)]),
 ((u'100', u'14381'), [(1.0, 8)])]

In [11]:
def compute_rec (x):
    num=sum(i[0]*i[1] for i in x[1])
    den=sum(i[0] for i in x[1])
    if den==0:
        return 0
    return x[0][0],(x[0][1],float(num)/den)





final_rec_rdd=rec_rdd.map(compute_rec)
final_rec_rdd.takeOrdered(10)

[(u'100', (u'10542', 5.0)),
 (u'100', (u'10825', 5.0)),
 (u'100', (u'11417', 1.0)),
 (u'100', (u'11940', 5.0)),
 (u'100', (u'12048', 5.0)),
 (u'100', (u'12255', 7.0)),
 (u'100', (u'13049', 4.0)),
 (u'100', (u'13190', 4.0)),
 (u'100', (u'14328', 9.0)),
 (u'100', (u'14381', 8.0))]

In [12]:
final_rec_rdd=final_rec_rdd.groupByKey()
final_rec_rdd.takeOrdered(10)

[(u'100', <pyspark.resultiterable.ResultIterable at 0x103c26b90>),
 (u'10001', <pyspark.resultiterable.ResultIterable at 0x106075310>),
 (u'10003', <pyspark.resultiterable.ResultIterable at 0x1060e3ed0>),
 (u'10011', <pyspark.resultiterable.ResultIterable at 0x106075e10>),
 (u'10050', <pyspark.resultiterable.ResultIterable at 0x1060e34d0>),
 (u'10056', <pyspark.resultiterable.ResultIterable at 0x1060e3550>),
 (u'10057', <pyspark.resultiterable.ResultIterable at 0x1060e3690>),
 (u'10065', <pyspark.resultiterable.ResultIterable at 0x1060e3290>),
 (u'10086', <pyspark.resultiterable.ResultIterable at 0x106075b50>),
 (u'10094', <pyspark.resultiterable.ResultIterable at 0x1060e3110>)]

In [13]:
final_rec_rdd=final_rec_rdd.map(lambda x: (x[0],list(y for y in x[1])))
final_rec_rdd.takeOrdered(10)

[(u'100',
  [(u'11940', 5.0),
   (u'7643', 7.0),
   (u'13049', 4.0),
   (u'353', 7.0),
   (u'2417', 8.0),
   (u'33638', 7.0),
   (u'29901', 4.0),
   (u'29389', 7.0),
   (u'7788', 5.0),
   (u'28030', 7.0),
   (u'12255', 7.0),
   (u'36341', 7.0),
   (u'28632', 6.0),
   (u'23334', 5.0),
   (u'5713', 8.0),
   (u'8013', 9.0),
   (u'27288', 5.0),
   (u'7999', 7.0),
   (u'30819', 6.0),
   (u'14660', 4.0),
   (u'31026', 6.0),
   (u'14328', 9.0),
   (u'16537', 5.0),
   (u'32397', 6.0),
   (u'11417', 1.0),
   (u'7679', 4.0),
   (u'3680', 7.0),
   (u'16502', 5.0),
   (u'16876', 8.0),
   (u'3497', 9.0),
   (u'15932', 6.0),
   (u'21986', 5.0),
   (u'6476', 6.0),
   (u'27966', 8.0),
   (u'30727', 7.0),
   (u'28964', 8.0),
   (u'5128', 6.0),
   (u'35432', 5.0),
   (u'26119', 7.0),
   (u'20003', 5.0),
   (u'16747', 7.0),
   (u'16394', 9.0),
   (u'18217', 7.0),
   (u'17812', 3.0),
   (u'26799', 5.0),
   (u'5850', 5.0),
   (u'12048', 5.0),
   (u'34832', 3.0),
   (u'8205', 9.0),
   (u'13190', 4.0),
   (u

In [14]:
ordered_final_rec_rdd=final_rec_rdd.map(lambda x:(x[0],sorted(x[1],key=lambda y: -y[1])))
ordered_final_rec_rdd.takeOrdered(10)

[(u'100',
  [(u'8013', 9.0),
   (u'14328', 9.0),
   (u'3497', 9.0),
   (u'16394', 9.0),
   (u'8205', 9.0),
   (u'32575', 9.0),
   (u'17815', 9.0),
   (u'33298', 9.0),
   (u'26529', 9.0),
   (u'26468', 9.0),
   (u'2417', 8.0),
   (u'5713', 8.0),
   (u'16876', 8.0),
   (u'27966', 8.0),
   (u'28964', 8.0),
   (u'2924', 8.0),
   (u'7811', 8.0),
   (u'16302', 8.0),
   (u'14381', 8.0),
   (u'22272', 8.0),
   (u'24512', 8.0),
   (u'7643', 7.0),
   (u'353', 7.0),
   (u'33638', 7.0),
   (u'29389', 7.0),
   (u'28030', 7.0),
   (u'12255', 7.0),
   (u'36341', 7.0),
   (u'7999', 7.0),
   (u'3680', 7.0),
   (u'30727', 7.0),
   (u'26119', 7.0),
   (u'16747', 7.0),
   (u'18217', 7.0),
   (u'34517', 7.0),
   (u'23506', 7.0),
   (u'36501', 7.0),
   (u'21358', 7.0),
   (u'3545', 7.0),
   (u'30386', 7.0),
   (u'28632', 6.0),
   (u'30819', 6.0),
   (u'31026', 6.0),
   (u'32397', 6.0),
   (u'15932', 6.0),
   (u'6476', 6.0),
   (u'5128', 6.0),
   (u'30360', 6.0),
   (u'32173', 6.0),
   (u'21336', 6.0),
   (u

In [15]:
reduced_final_rec_rdd=ordered_final_rec_rdd.map(lambda x:(x[0],x[1][0:5]))
reduced_final_rec_rdd.takeOrdered(10,key=lambda x:-int(x[0]))

[(u'15364',
  [(u'9922', 10.0),
   (u'4381', 10.0),
   (u'12769', 10.0),
   (u'34561', 10.0),
   (u'33601', 10.0)]),
 (u'15360',
  [(u'100', 10.0),
   (u'21918', 10.0),
   (u'19931', 10.0),
   (u'27280', 10.0),
   (u'33649', 10.0)]),
 (u'15338',
  [(u'22904', 9.0),
   (u'24372', 9.0),
   (u'26705', 9.0),
   (u'12467', 9.0),
   (u'17833', 9.0)]),
 (u'15337',
  [(u'29815', 10.0),
   (u'32530', 10.0),
   (u'18987', 10.0),
   (u'1760', 10.0),
   (u'32438', 10.0)]),
 (u'15331',
  [(u'23598', 10.0),
   (u'8394', 10.0),
   (u'22570', 10.0),
   (u'15743', 10.0),
   (u'15566', 10.0)]),
 (u'15329',
  [(u'19419', 10.0),
   (u'25656', 10.0),
   (u'31037', 10.0),
   (u'23030', 10.0),
   (u'20620', 10.0)]),
 (u'15328',
  [(u'35391', 10.0),
   (u'26681', 10.0),
   (u'27476', 10.0),
   (u'30072', 10.0),
   (u'14351', 10.0)]),
 (u'15316',
  [(u'9910', 10.0),
   (u'23384', 10.0),
   (u'35321', 10.0),
   (u'18973', 10.0),
   (u'4249', 10.0)]),
 (u'15313',
  [(u'18389', 10.0),
   (u'17407', 10.0),
   (u'1

In [16]:
def map_string (x):
    y=x[1]
    s=""
    for i in y:
        s+=str(i[0])+" "
    z=len(y)
    top=[33173, 33475, 35300, 1076, 31992]
    i=0
    while (z<5):
        s+=str(top[i])
        i+=1
        z+=1
    return x[0],s
recommendations_rdd=(reduced_final_rec_rdd
                     .map(map_string)
                     .takeOrdered(4143,key=lambda x: int(x[0])))

In [21]:
test_users=sc.textFile('test_no_header.csv')

test_list=test_users.collect()

recommended_users=list(i[0] for i in recommendations_rdd)
print test_list[0:9]
print recommendations_rdd[0:9]
print recommended_users[0:9]

[u'4', u'5', u'8', u'9', u'13', u'18', u'19', u'23', u'26']
[(u'5', '1076 12354 8519 1447 30137 '), (u'8', '18791 21043 4841 9404 1802 '), (u'18', '33400 17970 5525 23191 12001 '), (u'19', '18055 19425 636 9582 26117 '), (u'32', '6627 1617 28589 23571 20620 '), (u'35', '18303 12866 3077 27971 3431 '), (u'51', '1456 15823 28308 16891 23293 '), (u'53', '16501 20635 10701 35300 6839 '), (u'61', '11824 11873 17200 30904 5488 ')]
[u'5', u'8', u'18', u'19', u'32', u'35', u'51', u'53', u'61']


In [22]:
import csv
f1= open('submission_collaborative.csv','w+')
top="33173 33475 35300 1076 31992"


formatted_recommendations=recommendations_rdd;
csv_f1=csv.writer(f1)
csv_f1.writerow(["userId","testItems"])
i=0
j=0
while(i<4196):
    if (test_list[i] in recommended_users):
        csv_f1.writerow(formatted_recommendations[j])
        print formatted_recommendations[j]
        j+=1
    else:
        csv_f1.writerow((test_list[i],top))
        print (test_list[i],top)
    i+=1
len(recommended_users)

(u'4', '33173 33475 35300 1076 31992')
(u'5', '1076 12354 8519 1447 30137 ')
(u'8', '18791 21043 4841 9404 1802 ')
(u'9', '33173 33475 35300 1076 31992')
(u'13', '33173 33475 35300 1076 31992')
(u'18', '33400 17970 5525 23191 12001 ')
(u'19', '18055 19425 636 9582 26117 ')
(u'23', '33173 33475 35300 1076 31992')
(u'26', '33173 33475 35300 1076 31992')
(u'29', '33173 33475 35300 1076 31992')
(u'31', '33173 33475 35300 1076 31992')
(u'32', '6627 1617 28589 23571 20620 ')
(u'33', '33173 33475 35300 1076 31992')
(u'35', '18303 12866 3077 27971 3431 ')
(u'37', '33173 33475 35300 1076 31992')
(u'51', '1456 15823 28308 16891 23293 ')
(u'52', '33173 33475 35300 1076 31992')
(u'53', '16501 20635 10701 35300 6839 ')
(u'54', '33173 33475 35300 1076 31992')
(u'61', '11824 11873 17200 30904 5488 ')
(u'64', '7597 30653 9913 15743 12866 ')
(u'69', '33173 33475 35300 1076 31992')
(u'76', '33475 24758 13069 27000 10082 ')
(u'78', '33173 33475 35300 1076 31992')
(u'79', '33173 33475 35300 1076 31992')
(

1986